# co-visitation matrix_time_weighted
時間差の重み付ける  
各sessionのtailのみ使用

In [7]:
import os
import sys
import gc
import subprocess
from dotenv import load_dotenv
load_dotenv
sys.path.append(os.getenv('UTILS_PATH'))
from tqdm import tqdm
import multiprocessing
import random
from collections import defaultdict

import pandas as pd
import cudf
import matplotlib.pyplot as plt

In [8]:
SEED = 42
random.seed(SEED)

In [9]:
INPUT_DIR = os.getenv('INPUT_DIR')
OUTPUT_DIR = os.getenv('OUTPUT_DIR')
PREP_DIR = os.getenv("PREP_DIR")

In [10]:
CHUNK_N = 400
W = 3

In [11]:
week = "week3"

In [12]:
# データ読み込み
files = [
    "train_sessions_week1.pkl",
    "test_sessions_week1.pkl",
    "train_sessions_week2.pkl",
    "test_sessions_week2.pkl",
    "train_sessions_week3.pkl",
#    "test_sessions_week3.pkl",
    "train_sessions_week4.pkl",
    "test_sessions_week4.pkl",
    "test_sessions.pkl"
]

dfs = []

for file in files:
    dfs.append(pd.read_pickle(PREP_DIR + file))
sessions = pd.concat(dfs)

sessions = sessions.drop(columns=["type"])
sessions["ts"] = (sessions["ts"]/1000).astype("int32")

# chunkのグループ分け
sessions["chunk"] = sessions["session"] % CHUNK_N

In [13]:
dfs_outer = []
dfs_inner = []

pair_df = pd.DataFrame(columns=["aid_x", "aid_y", "cnt"])
for i, chunk_df in tqdm(sessions.groupby("chunk"), total=CHUNK_N):
    df = cudf.from_pandas(chunk_df)
    df = df.reset_index(drop=True)
    df = df.sort_values(["session", "ts"], ascending=[True, False])
    df["n"] = df.groupby("session").cumcount()
    df = df.loc[df.n<30].drop("n", axis=1)
    df = df.merge(df, on="session")
    df = df.loc[((df.ts_x - df.ts_y).abs() < 24 * 60 * 60) & (df.aid_x != df.aid_y)]
    df["wt"] = (W - 1) * (1 - (df.ts_x - df.ts_y).abs() / (24*60*60)) + 1
    df = df.sort_values("wt").drop_duplicates(subset=["session", "aid_x", "aid_y"], keep="last")
    df = df.groupby(["aid_x", "aid_y"])["wt"].sum().reset_index()
    df = df.sort_values(["aid_x", "wt"], ascending=(True, False))
    df = df[df.groupby("aid_x").cumcount()<30]
    dfs_inner.append(df.to_pandas())

    if i % 100 == 99:
        if len(dfs_inner) > 0:
            pair_df_inner = pd.concat(dfs_inner)
            pair_df_inner = pair_df_inner.groupby(["aid_x", "aid_y"])["wt"].sum().reset_index()
            dfs_outer.append(pair_df_inner)
            dfs_inner = []
            del pair_df_inner
            gc.collect()

if len(dfs_inner) > 0:
    pair_df_inner = pd.concat(dfs_inner)
    pair_df_inner = pair_df_inner.groupby(["aid_x", "aid_y"])["wt"].sum().reset_index()
    dfs_outer.append(pair_df_inner)
    del pair_df_inner
    gc.collect()

del dfs_inner
gc.collect()

pair_df = pd.concat(dfs_outer)
pair_df = pair_df.groupby(["aid_x", "aid_y"])["wt"].sum().reset_index()
pair_df = pair_df.sort_values(["aid_x", "wt"], ascending=(True, False))
pair_df = pair_df.groupby("aid_x").head(20)
pair_df.to_pickle(PREP_DIR + f"co_visitation_matrix_time_weighted_v2_{week}.pkl")

100%|██████████| 400/400 [04:50<00:00,  1.38it/s]  
